In [1]:
import pandas as pd
import re
import sys
import numpy as np
import nltk
from nltk.corpus import wordnet as wn
from nltk import pos_tag
from nltk.tag import UnigramTagger
from nltk.corpus import brown
import webcolors
from IPython.display import display
sys.path.append("..")
from ADA_JEX2017.Project.Functions.str_functions import *
from ADA_JEX2017.Project.Functions.pre_process import *


In [2]:
path='../ADA_JEX2017/Project/Functions/'

In [3]:
#our datafile exported to csv
data_file='./recipeInfo/recipeInfo_WestWhiteHorvitz_WWW2013.csv'
raw_data = pd.read_csv(data_file ,sep=';')

In [4]:
with open(path+'units_list.txt', 'r') as f:
    units_list = [line.rstrip('\n') for line in f]
    
with open(path+'technique_list.txt', 'r') as f:
    techniques_list = [line.rstrip('\n') for line in f]

with open(path+'ingredient_list.txt', 'r') as f:
    ingredient_list = [line.rstrip('\n') for line in f]
    
lemmatizer = WordNetLemmatizer()
ingredient_list=[lemmatizer.lemmatize(token).lower() for token in ingredient_list]

In [5]:
ingr_dataframe=raw_data[['title','ingredients_list','ingredients_bag-of-words']].copy().dropna()
ingr_dataframe = ingr_dataframe.reset_index(drop=True)
display(ingr_dataframe.head())
ingr_data_reduced=ingr_dataframe.head(100)
#we ignore upper case in the ingredients string
ingr_dataframe['ingredients_list']=ingr_dataframe['ingredients_list'].str.lower()

,title,ingredients_list,ingredients_bag-of-words
0,Easy Light Chocolate Milkshake Recipe,"put one half cup of milk, 4 tablespoons of cho...","! , . 4 ? and are blend chocolate cup enjoy ex..."
1,Lamb Stew Recipe : : Recipes : Food Network,5 yellow onions|2 turnips|5 carrots|1 stalk fe...,", 1 1\/2 1\/4 1\/8 1Â 1\/2 2 3 5 and anise app..."
2,Chocolate Bread Pudding Recipe : Paula Deen : ...,"1 (1-pound) loaf French or Italian bread, cube...",", -lrb- -rrb- 1 1-pound 1\/2 1\/4 1Â 1\/2 2 3 ..."
3,Snowball Cookies II Recipe,1/2 cup powdered sugar|1/3 cup butter or marga...,1 1\/2 1\/3 2 almond baking bisquick butter ch...
4,Old Fashioned Butterscotch Pie Recipe #31698,1 1/2 cup brown sugar|1 cup water|3 eggs|4 tab...,1 1\/2 1\/4 1Â 1\/2 3 4 brown butter cream cup...


In [6]:
fun_add_preprocess(ingr_dataframe,units_list)

In [ ]:
#-----test cell : test a specific receipe -------------
receipe=ingr_dataframe.loc[0]['Recipe_preporcess']
print(receipe)
dic_ingr,dictec,wasted,wasted_numb=fun_extract_ingredients\
    (receipe,ingredient_list,techniques_list,units_list,to_gram=True)

In [7]:
#----- calculate whole data frame ---------
ingr_data_reduced=ingr_dataframe.head(100)
all_dic=[]
not_used_ingr=[]
wastes=0
for index, row in ingr_dataframe.iterrows():
    recipe=row['Recipe_preporcess']
    dic_ingre,dictec,wasted_ingr,wasted_number=fun_extract_ingredients\
            (recipe,ingredient_list,techniques_list,units_list,to_gram=True)
    all_dic.append(dic_ingre)
    not_used_ingr.append(wasted_ingr)
    wastes=wastes+wasted_number

In [8]:
# --- actually create the dataframe
ingredients_frame=pd.DataFrame(data=all_dic)
display(ingredients_frame.head(5))

,acidulated water,acorn squash,agave,ale,aleppo pepper,allspice,almond,almond extract,almond milk,amaranth,...,worcestershire sauce,wrap,yam,yeast,yellow lentil,yoghurt,yogurt,yuzu,zest,zucchini
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,237.0,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
print('There are : ',len(list(ingredients_frame)), 'ingredients')
ingred_used={}
for i in list(ingredients_frame):
    ingred_used[i]=sum(ingredients_frame[i].value_counts())

There are :  812 ingredients


In [11]:
occu=sorted((value,key) for (key,value) in ingred_used.items())
occu[::-1]

[(20023, 'salt'),
 (14109, 'onion'),
 (13848, 'sugar'),
 (12742, 'butter'),
 (11656, 'flour'),
 (11280, 'egg'),
 (10491, 'pepper'),
 (9826, 'garlic'),
 (9653, 'water'),
 (8790, 'black pepper'),
 (8714, 'olive oil'),
 (7060, 'milk'),
 (5687, 'tomato'),
 (5011, 'cream'),
 (4886, 'brown sugar'),
 (4729, 'vanilla extract'),
 (4624, 'vegetable oil'),
 (4371, 'cinnamon'),
 (4361, 'clove'),
 (4205, 'parsley'),
 (3817, 'lemon juice'),
 (3795, 'sauce'),
 (3602, 'baking powder'),
 (3529, 'broth'),
 (3301, 'soda'),
 (3277, 'carrot'),
 (3070, 'celery'),
 (2809, 'potato'),
 (2765, 'cheese'),
 (2532, 'cream cheese'),
 (2515, 'chicken breast'),
 (2500, 'basil'),
 (2425, 'parmesan cheese'),
 (2415, 'thyme'),
 (2409, 'oregano'),
 (2280, 'cheddar cheese'),
 (2265, 'ginger'),
 (2227, 'vinegar'),
 (2218, 'beef'),
 (2192, 'margarine'),
 (2175, 'oil'),
 (2157, 'bread'),
 (2134, 'mushroom'),
 (2062, 'nutmeg'),
 (2008, 'soy sauce'),
 (1936, 'cumin'),
 (1934, 'bean'),
 (1927, 'seasoning'),
 (1916, 'cilantro'),

## Co-occurences and Covariance

#### NearestNeighbors: 

calculate for each point the number of Neighbors in a hypersphere of radius r.

In [12]:
from sklearn.neighbors import NearestNeighbors

In [19]:
newdf = ingredients_frame.notnull().astype('int')
coocc = newdf.T.dot(newdf)

,acidulated water,acorn squash,agave,ale,aleppo pepper,allspice,almond,almond extract,almond milk,amaranth,...,worcestershire sauce,wrap,yam,yeast,yellow lentil,yoghurt,yogurt,yuzu,zest,zucchini
acidulated water,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
acorn squash,0,65,0,0,0,1,1,0,0,0,...,0,0,1,0,0,0,1,0,0,0
agave,0,0,50,0,0,1,6,2,1,1,...,1,0,0,2,0,0,1,0,0,2
ale,0,0,0,17,0,0,2,0,0,0,...,3,0,0,0,0,0,0,0,1,0
aleppo pepper,0,0,0,0,4,0,0,0,0,0,...,2,0,0,0,0,0,0,0,0,0
allspice,0,1,1,0,0,604,17,5,0,0,...,26,0,3,4,0,0,9,0,25,11
almond,0,1,6,2,0,17,1065,93,3,0,...,18,0,0,15,0,0,31,0,47,9
almond extract,0,0,2,0,0,5,93,442,1,0,...,0,0,0,7,0,0,6,0,12,1
almond milk,0,0,1,0,0,0,3,1,32,0,...,0,0,0,3,0,0,1,0,1,0
amaranth,0,0,1,0,0,0,0,0,0,3,...,0,0,0,0,0,0,0,0,0,0


#### Similarities between ingredients:

In [74]:
r=0.4
number_neighbors_toshow=5
neigh = NearestNeighbors(number_neighbors_toshow,r)
neigh.fit(coocc.values)

NearestNeighbors(algorithm='auto', leaf_size=30, metric='minkowski',
         metric_params=None, n_jobs=1, n_neighbors=5, p=2, radius=0.4)

In [87]:
b = neigh.kneighbors(coocc.values, return_distance=False)
a = pd.DataFrame(b)
d = dict(zip(range(len(ingredients_frame.columns)),list(ingredients_frame.columns)))
a.replace(d, inplace=True)
a.rename(index=d,inplace=True)
a.loc['apple chutney']

0    apple chutney
1        lamb rack
2      ginger beer
3       herbal tea
4    clotted cream
Name: apple chutney, dtype: object

#### check similiraties between Recipes:

In [17]:
r=0.4
number_neighbors_toshow=5
neigh = NearestNeighbors(number_neighbors_toshow,r)
neigh.fit(newdf.values)

NearestNeighbors(algorithm='auto', leaf_size=30, metric='minkowski',
         metric_params=None, n_jobs=1, n_neighbors=5, p=2, radius=0.4)

In [18]:
b = neigh.kneighbors(coocc.values, return_distance=False)
a = pd.DataFrame(b)
d = dict(zip(range(len(ingredients_frame.columns)),list(ingredients_frame.columns)))
a.replace(d, inplace=True)
a.rename(index=d,inplace=True)
a.loc['chicken']

0    31128
1     2688
2    43121
3    31084
4    46156
Name: chicken, dtype: object

#### Covariance

C_ij is the covariance between the i th and j th ingredients.

Covariance is just unscaled correlation.  
If a number at a certain position in the covariance matrix is large, then the variable that corresponds to that row and the variable that corresponds to that column change with one another. When one goes up, the other goes up. When one goes down, the other goes down. 

In [45]:
ingr_matrix=ingredients_frame.fillna(0).values.T
print((ingr_matrix).shape)
M=(ingr_matrix).shape[0]

(812, 46277)


In [41]:
# covariance matrix
X=ingr_matrix-(1/M)*np.dot(np.ones((M,M)),ingr_matrix)
coco=np.dot(X.T,X)/M

KeyboardInterrupt: 

In [47]:
from sklearn.covariance import shrunk_covariance,ledoit_wolf
C_lw=ledoit_wolf(ingr_matrix)

KeyboardInterrupt: 

In [42]:
covar=np.cov(ingr_matrix)

In [46]:
covar

array([[  7.77924239e-04,  -1.30281633e-05,  -4.58283155e-05, ...,
         -2.80175555e-09,  -3.24313291e-05,  -8.07677202e-03],
       [ -1.30281633e-05,   1.13283057e+02,  -3.55169445e-02, ...,
         -2.17136055e-06,  -2.51342801e-02,  -6.25949831e+00],
       [ -4.58283155e-05,  -3.55169445e-02,   9.46539318e+02, ...,
         -7.63805259e-06,  -8.84132084e-02,  -2.20177623e+01],
       ..., 
       [ -2.80175555e-09,  -2.17136055e-06,  -7.63805259e-06, ...,
          2.16090066e-05,  -5.40522152e-06,  -1.34612867e-03],
       [ -3.24313291e-05,  -2.51342801e-02,  -8.84132084e-02, ...,
         -5.40522152e-06,   1.45317175e+01,  -1.51894527e+01],
       [ -8.07677202e-03,  -6.25949831e+00,  -2.20177623e+01, ...,
         -1.34612867e-03,  -1.51894527e+01,   1.51696566e+08]])

In [48]:
d = dict(zip(range(len(ingredients_frame.columns)),list(ingredients_frame.columns)))


In [78]:
i=15
print('With: ',d[i])
arr = np.array(covar[i])
topingr=arr.argsort()[-6:][::-1]
print('you should try:')
for x in topingr[:]:
    print(d[x])

With:  apple chutney
you should try:
pork
apple chutney
cider
red wine
thyme
clove
